# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!
  

- 🤝 Breakout Room #2:
  1. Creating an Evaluation Dataset
  2. Adding Evaluators
  3. Evaluating

# 🤝 Breakout Room #1

## LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effetively allowing us to recreate appliation flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

In [98]:
!pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [99]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [100]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE1 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [116]:

from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun



tool_belt = [
    DuckDuckGoSearchRun(description='A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current topics. Input should be a search query.'),
    ArxivQueryRun(description='A wrapper around Arxiv Search. Useful for when you need to answer questions of scientific or technical nature. Input should be a search query.'),
]

### Actioning with Tools

Now that we've created our tool belt - we need to create a process that will let us leverage them when we need them.

We'll use the built-in [`ToolExecutor`](https://github.com/langchain-ai/langgraph/blob/main/langgraph/prebuilt/tool_executor.py) to do so.

In [117]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [118]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0)

#from langchain.agents import initialize_agent, load_tools

#tools = load_tools(["dalle-image-generator"])
#agent = initialize_agent(tools, model, agent="zero-shot-react-description", verbose=True)
#output = agent.run("Create an image of a halloween night at a haunted museum")

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [119]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

#### ❓ Question #1:

How does the model determine which tool to use?

**Answer**: The LLM (which is the agent) has access to a description of the tools. It directs a specific query to a tool (description) through a process not completely dissimilar to a classification problem. 

## Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [120]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

## It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [121]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `call_tool` is a node which will call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [122]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [123]:
workflow.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [124]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [125]:
workflow.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [126]:
app = workflow.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

**Answer**: There is no specific limit to how many times we can cycle. To impose a limit on the number of cycles we could add a conditional statement inside the "should_continue" function:


In [127]:
def capped_should_continue(state):

    last_message = state["messages"][-1]
    
    max_cycles = 3 #we choose 3 for the max number of cycles as an example
    m = len(state["messages"])
    n = 2*max_cycles

    if "function_call" not in last_message.get('additional_kwargs', {}) or m > n:
        return "end"

    return "continue"

This uses the length of state["messages"] (m) as a way to track how many cycles have already happened. If m is greater than a certain number, the cycles are interrupted. 

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [130]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is RAG in the context of Large Language Models? When did it break onto the scene?")]}

app.invoke(inputs)


{'messages': [HumanMessage(content='What is RAG in the context of Large Language Models? When did it break onto the scene?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"RAG in the context of Large Language Models"}', 'name': 'duckduckgo_search'}}, response_metadata={'finish_reason': 'function_call', 'logprobs': None}),
  FunctionMessage(content='Retrieval-augmented generation (RAG) is a technique used to "ground" large language models (LLMs) with specific data sources, often sources that weren\'t included in the models\' original ... RAG stands for R etrieval- A ugmented G eneration. RAG enables large language models (LLM) to access and utilize up-to-date information. Hence, it improves the quality of relevance of the response from LLM. Below is a simple diagram of how RAG is implemented. Key Takeaways. RAG is a relatively new artificial intelligence technique that can improve the quality of generative AI by allowing large language model (LLMs)

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "function_call" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "function_call" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [131]:
#inputs = {"messages" : [HumanMessage(content="What is QLoRA in Machine Learning? Are their any papers that could help me understand? Once you have that information, can you look up the bio of the first author on the QLoRA paper?")]}

#app.invoke(inputs)

####🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

**Answer**:

1) The query was received
2) The agent delegated to DuckduckGo a search about QLoRA in machine learning.
3) The agent received feedback from DuckDuckGo and further proceeded to send a request to Arxiv in order to solve another part of the original query (papers about QLoRA)
4) The agent receives feedback from Arxiv and realises the last part of the query was still not satisfied, i.e., the bio of the first author of the QLoRA paper, so it sends a new request to DuckDuckGo
5) The agent receives new feedback from DuckDuckGo and as all information has been gathered, proceeds to write a final answer to the user. 

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [132]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | app | parse_output

In [133]:
agent_chain.invoke({"question" : "What is RAG?"})

"RAG stands for Retrieval-augmented generation. It is an AI framework that improves the quality of language model-generated responses by grounding the model on external sources of knowledge to supplement the model's internal representation of information. RAG combines information retrieval with text generation, allowing AI models to create contextually accurate and information-rich text."

# 🤝 Breakout Room #2

## Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

####🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [134]:
questions = [
    "What is lipid hopping?",
    "How did a germanic language like Old-English evolved to be so heavily influenced by Latin?",
    "Who is Taylor Swift?",
    "What is STED microscopy and who invented it?",
    "Which countries have a GDP higher than the market capitalization of Microsoft and Apple combined?"
    ### YOUR QUESTIONS HERE
]

answers = [
    {"must_mention" : ["plasma membrane", "diffusion"]},
    {"must_mention" : ["Norman", "French"]},
    {"must_mention" : ["American", "singer"]},
    {"must_mention" : ["diffraction", "Hell"]},
    {"must_mention" : ["United States", "China"]}
    ### YOUR ANSWERS HERE
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [135]:
from langsmith import Client

client = Client()
dataset_name = f"Agentic use of tools - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about a variety of topics to evaluate our newly created agent."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

**Answer**: Correct answers are associated with the questions via their correspondent "must mention" associated pair of strings. It is assumed that a correct answer to each given question must necessarily feature the "must mention" par of strings. This is problematics as there are usually many ways to correctly answer a question, and each such possible correct answer can further be reworded in many ways. 

> NOTE: Feel free to indicate if this is problematic or not

## Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [136]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

**Answer**: As discussed in Question 3, the "must mention" method is problematic because some questions may have answers that are correct without needing to explicitly mention all the "must_mention" terms. This method might unfairly penalize nuanced answers that convey the necessary information through an analysis that doesn't directly use the specified terms.

One way to address this problem would be to add more terms to the pool of "must mention" phrases and consider correct an answer that mentions at least one of them. 

Now that we have created our custom evaluator - let's initialize our `RunEvalConfig` with it, and a few others:

- `"criteria"` includes the default criteria which, in this case, means "helpfulness"
- `"cot_qa"` includes a criteria that bases whether or not the answer is correct by utilizing a Chain of Thought prompt and the provided context to determine if the response is correct or not.

In [137]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
    evaluators=[
        "criteria",
        "cot_qa",
    ],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [138]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_chain,
    evaluation=eval_config,
    verbose=True,
    project_name=f"Agentic Pipeline - Evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'Agentic Pipeline - Evaluation - 6abefdf8' at:
https://smith.langchain.com/o/502cb790-b0a2-5680-b609-70ababf6c8f1/datasets/1370384f-d692-4a63-8b6c-b70b721d477c/compare?selectedSessions=b2979362-75db-41d6-8b94-97f5a412002a

View all tests for Dataset Agentic use of tools - Evaluation Dataset - caf0a297 at:
https://smith.langchain.com/o/502cb790-b0a2-5680-b609-70ababf6c8f1/datasets/1370384f-d692-4a63-8b6c-b70b721d477c
[------------------------------------------------->] 5/5

,feedback.helpfulness,feedback.COT Contextual Accuracy,feedback.must_mention,error,execution_time,run_id
count,5.0,5.000000,5,0,5.000000,5
unique,NaN,NaN,1,0,NaN,5
top,NaN,NaN,False,NaN,NaN,edfc5d39-fee5-41c2-bb8e-bcd1da581f0f
freq,NaN,NaN,5,NaN,NaN,1
mean,1.0,0.200000,NaN,NaN,5.205691,NaN
std,0.0,0.447214,NaN,NaN,0.900292,NaN
min,1.0,0.000000,NaN,NaN,4.249592,NaN
25%,1.0,0.000000,NaN,NaN,4.892358,NaN
50%,1.0,0.000000,NaN,NaN,5.034130,NaN
75%,1.0,0.000000,NaN,NaN,5.163909,NaN


{'project_name': 'Agentic Pipeline - Evaluation - 6abefdf8',
 'results': {'4d31abb3-dbc3-46e9-a672-50a8c1f126c6': {'input': {'question': 'What is lipid hopping?'},
   'feedback': [EvaluationResult(key='helpfulness', score=1, value='Y', comment='The criterion for this task is "helpfulness". \n\n1. The submission provides a detailed explanation of what lipid hopping is, which is helpful for someone who is looking for an answer to this question. \n\n2. The submission goes beyond just defining lipid hopping, it also explains the role it plays in lipid metabolism and energy homeostasis, which adds insight to the answer.\n\n3. The submission is appropriate as it directly answers the question and does not include any irrelevant or inappropriate information.\n\nBased on these points, the submission meets the criterion of being helpful, insightful, and appropriate.\n\nY', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('05d5731a-cb2f-4a1a-815c-7cd0833e52fb'))}, source_run_id=None,